<a href="https://colab.research.google.com/github/RuiqiTang/Learning-Diffusion/blob/main/ddim_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/RuiqiTang/Learning-Diffusion.git

fatal: destination path 'Learning-Diffusion' already exists and is not an empty directory.


In [11]:
cd Learning-Diffusion

[Errno 2] No such file or directory: 'Learning-Diffusion'
/content/Learning-Diffusion


In [13]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from torchvision import datasets,transforms

from models.UNet2 import UNetModel
from DDIM.ddim import GaussianDiffusion
from Utils.plotting import plot_image_grid

In [14]:
batch_size=64
timesteps=500
epochs=10
device='cuda' if torch.cuda.is_available() else 'cpu'


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [15]:
dataset=datasets.MNIST('/Data',train=True,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)

In [16]:
# Load: Model
model=UNetModel(
    in_channels=1,
    model_channels=96,
    out_channels=1,
    channel_mult=(1,2,2),
    attention_resolutions=[]
)
model.to(device)

gaussian_diffusion=GaussianDiffusion(timesteps=timesteps)
optimizer=torch.optim.Adam(model.parameters(),lr=5e-4)

In [17]:
# Train Process
images:torch.Tensor
epoch_loss:float

wandb.init(
    project='ddim-mnist',
    config={
        "batch_size":batch_size,
        "timesteps":timesteps,
        "epochs":epochs,
        "lr":5e-4,
        "model_channels":96,
    }
)
wandb.watch(model,log='all')

for epoch in range(epochs):
    epoch_loss=0.0
    for step,(images,labels) in tqdm(enumerate(train_loader),desc=f'Epoch:{epoch}'):
        optimizer.zero_grad()
        batch_size=images.shape[0]
        images=images.to(device)

        '''
            为每个时间步均匀地采样一个时间步
            每个样本需要在不同的时间步上进行加噪或去噪，通过均匀采样t可以让模型学习到所有时间步的特征，确保对不同程度的噪声都有泛化能力
        '''
        t=torch.randint(0,timesteps,(batch_size,),device=device).long()
        loss=gaussian_diffusion.train_losses(model,images,t)

        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
        global_step=epoch*len(train_loader)+step
        wandb.log({
            'train/loss':loss.item(),
            'epoch':epoch,
            'step':global_step
        })

    avg_loss=epoch_loss/len(train_loader)
    wandb.log({'train/epoch_loss':avg_loss,'epoch':epoch})



/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ruiqitarng (ruiqitarng-fudan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch:0: 938it [03:25,  4.56it/s]
Epoch:1: 938it [03:26,  4.55it/s]
Epoch:2: 938it [03:25,  4.57it/s]
Epoch:3: 938it [03:25,  4.57it/s]
Epoch:4: 938it [03:25,  4.57it/s]
Epoch:5: 938it [03:25,  4.57it/s]
Epoch:6: 938it [03:25,  4.56it/s]
Epoch:7: 938it [03:25,  4.56it/s]
Epoch:8: 938it [03:25,  4.55it/s]
Epoch:9: 938it [03:25,  4.57it/s]
